GBIF DISTRIBUTION AGGREGATION

In [1]:
import numpy as np
import pandas as pd
import os.path
import warnings

In [2]:
checklist = pd.read_csv('/home/joon/data/checklist_taxonomy_preprocessed.csv',index_col='accepted_plant_name_id')
checklist = checklist[['genus','species', 'plant_name_id']]
checklist.dropna(axis=0,subset=['species'],inplace=True)
checklist_genus_values = checklist.genus.unique()

# Helper functions

In [3]:
# def gbif_to_datetime(gbif):
#     return pd.to_datetime(gbif.loc[~gbif.day.isnull(),['year','month','day']])

In [4]:
# HELPER FUNCTION
def gbif_n_obs_last_m_years(gbif,years):
    return np.sum(np.histogram(gbif.year,bins=range(1500,2020))[0][-years:])

# Functions

In [5]:
def gbif_n_institutionCode(gbif):
    return gbif.institutionCode.nunique()

In [6]:
def gbif_n_recordedBy(gbif):
    return gbif.recordedBy.nunique()

In [7]:
def gbif_n_obs(gbif):
    return gbif.shape[0]

In [8]:
def gbif_n_country(gbif):
    return gbif['countryCode'].nunique()

In [9]:
def gbif_max_obs_date(gbif):
    return np.max(gbif.year)

In [10]:
def gbif_min_obs_date(gbif):
    return np.min(gbif.year)

In [11]:
def gbif_delta_obs_date(gbif):
    return np.max(gbif.year)-np.min(gbif.year)

In [12]:
def gbif_age_last_obs(gbif):
    return 2019-np.max(gbif.year)

In [13]:
def gbif_age_first_obs(gbif):
    return 2019-np.min(gbif.year)

In [14]:
date_hist_bins = range(1700,2021,20)
def gbif_date_hist(gbif):
    return np.histogram(gbif.year,bins=date_hist_bins)[0] 

## Number of observations in last: 5 years, 10 years, 20 years, 50 years, all time

In [15]:
def gbif_n_obs_last_5y(gbif):
    return gbif_n_obs_last_m_years(gbif,5)

In [16]:
def gbif_n_obs_last_10y(gbif):
    return gbif_n_obs_last_m_years(gbif,10)

In [17]:
def gbif_n_obs_last_20y(gbif):
    return gbif_n_obs_last_m_years(gbif,20)

In [18]:
def gbif_n_obs_last_50y(gbif):
    return gbif_n_obs_last_m_years(gbif,50)

# Latitude: min/max/mean/median

In [19]:
def gbif_min_latitude(gbif):
    return np.min(np.abs(gbif.decimalLatitude))

In [20]:
def gbif_max_latitude(gbif):
    return np.max(np.abs(gbif.decimalLatitude))

In [21]:
def gbif_mean_latitude(gbif):
    return np.mean(np.abs(gbif.decimalLatitude))

In [22]:
def gbif_mean_pos_latitude(gbif):
    return np.mean(gbif.decimalLatitude[gbif.decimalLatitude>=0])

In [23]:
def gbif_mean_neg_latitude(gbif):
    return np.mean(gbif.decimalLatitude[gbif.decimalLatitude<=0])

In [24]:
def gbif_median_latitude(gbif):
    foo = gbif.decimalLatitude.dropna()
    return np.median(np.abs(foo)) if not foo.empty else np.nan

In [25]:
def gbif_latitude_hist(gbif):
    return np.histogram(gbif.decimalLatitude,bins=range(-90,91))[0]

In [26]:
def gbif_n_tropical_latitudes(gbif):
    return gbif.decimalLatitude[np.abs(gbif.decimalLatitude)<23.43].shape[0]

## Convex hull surface (extent of occurence, EOO)

In [27]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPoint, Point
from shapely.ops import cascaded_union

In [28]:
country_polygons = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')).geometry
land_polygon = cascaded_union(country_polygons)

In [29]:
import pyproj    
import shapely
import shapely.ops as ops
from shapely.geometry.polygon import Polygon
from functools import partial
import warnings

def earth_polygon_area(polygon):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        geom_area = ops.transform(
            partial(
                pyproj.transform,
                pyproj.Proj(init='EPSG:4326'),
                pyproj.Proj(
                    proj='aea',
                    lat_1=polygon.bounds[1],
                    lat_2=polygon.bounds[3])),
            polygon)
    return geom_area.area

In [30]:
def gbif_eoo(gbif):
    global land_polygon
    obs_locations = gbif.loc[(~gbif.decimalLatitude.isnull() & ~gbif.decimalLongitude.isnull()),['decimalLongitude','decimalLatitude']].values
    if obs_locations.shape[0] <= 2:
        return np.nan
    else:
        convex_hull = MultiPoint(obs_locations).convex_hull
        convex_hull_cap_land = convex_hull.intersection(land_polygon)
        try:
            area = earth_polygon_area(convex_hull_cap_land)
        except:
            return np.nan
        return area

# Area of occupancy (AOO) (union of 2km grid around occurences)

In [31]:
import math

def grid_around(lon,lat):
    delta_lat = 90/10000
    delta_lon = 90/10000/math.cos(lat*2*3.14159/360)

    polygon = [(lon-delta_lon,lat-delta_lat),(lon+delta_lon,lat-delta_lat),(lon+delta_lon,lat+delta_lat),(lon-delta_lon,lat+delta_lat)]
    return Polygon(polygon)

In [32]:
def gbif_aoo(gbif):
    obs_locations = gbif.loc[(~gbif.decimalLatitude.isnull() & ~gbif.decimalLongitude.isnull()),['decimalLongitude','decimalLatitude']].values
    polygons = [grid_around(lon,lat) for [lon,lat] in obs_locations]
    union = cascaded_union(polygons)
    try:
        area = earth_polygon_area(union)
    except:
        return np.nan
    return area

# Number of connected components ... ?
TODO:

# Protected areas

# Geographic distribution of observations: histogram (for DL)

In [33]:
hist2d_resolution = 100

In [34]:
def gbif_obs_geographic_distr_hist(gbif):
    obs_locations = gbif.loc[(~gbif.decimalLatitude.isnull() & ~gbif.decimalLongitude.isnull()),['decimalLongitude','decimalLatitude']].values
    return np.histogram2d(obs_locations[:,0],obs_locations[:,1],[np.linspace(-180,180,num=hist2d_resolution+1),np.linspace(-90,90,num=hist2d_resolution+1)])[0].flatten()

## Bioclim variables

In [35]:
bioclim = pd.read_csv('/home/joon/data/bioclim.csv',index_col=0)

/usr/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [36]:
X_bioclim_values = bioclim.X.unique()
Y_bioclim_values = bioclim.Y.unique()

In [37]:
bioclim.set_index(['X','Y'],inplace=True)

In [38]:
def match_coords_to_grid(coords,grid):
    coords = np.asarray(coords)
    return grid[np.abs((coords.reshape((len(coords),1))-grid)).argmin(axis=1)]

In [39]:
def gbif_obs_bioclim(gbif):
    global X_bioclim_values, Y_bioclim_values, bioclim
    gbif = gbif.loc[(~gbif.decimalLatitude.isnull() & ~gbif.decimalLongitude.isnull()),['decimalLongitude','decimalLatitude']]
    gbif['X'] = match_coords_to_grid(gbif.decimalLongitude,X_bioclim_values)
    gbif['Y'] = match_coords_to_grid(gbif.decimalLatitude,Y_bioclim_values)
    bioclim_selection = bioclim.loc[[tuple(x) for x in gbif[['X','Y']].values]]
    
    return list(np.mean(bioclim_selection))

## Climate change {Temperature, rain}: diff(50 years ago,today)
TODO:

## Population density:

In [40]:
pop_density_1975 = pd.read_csv('/home/joon/data/ghs_pop_1975.xyz',header=None,names=['longitude','latitude','density'],sep=' ')

In [41]:
pop_density_2015 = pd.read_csv('/home/joon/data/ghs_pop_2015.xyz',header=None,names=['longitude','latitude','density'],sep=' ')

In [42]:
pop_density_1975.density.replace(to_replace=-200.0,value=np.nan,inplace=True)
pop_density_2015.density.replace(to_replace=-200.0,value=np.nan,inplace=True)

In [43]:
longitude_values = pop_density_2015.longitude.unique()
latitude_values = pop_density_2015.latitude.unique()

In [44]:
pop_density_1975.set_index(['longitude','latitude'],inplace=True)
pop_density_2015.set_index(['longitude','latitude'],inplace=True)

In [45]:
def gbif_pop_density(gbif):
    global longitude_values, latitude_values
    gbif = gbif.loc[(~gbif.decimalLatitude.isnull() & ~gbif.decimalLongitude.isnull()),['decimalLongitude','decimalLatitude']]
    gbif['longitude'] = match_coords_to_grid(gbif.decimalLongitude,longitude_values)
    gbif['latitude'] = match_coords_to_grid(gbif.decimalLatitude,latitude_values)
    pop_density_1975_selection = pop_density_1975.loc[[tuple(x) for x in gbif[['longitude','latitude']].values]]
    pop_density_2015_selection = pop_density_2015.loc[[tuple(x) for x in gbif[['longitude','latitude']].values]]
    pop_density_1975_mean = np.mean(pop_density_1975_selection)[0]
    pop_density_2015_mean = np.mean(pop_density_2015_selection)[0]

    return [pop_density_1975_mean,pop_density_2015_mean,pop_density_2015_mean-pop_density_1975_mean]

# basis of record

In [46]:
gbif_basisOfRecord_values = pd.read_csv('/home/joon/data/gbif-basisOfRecord-values.csv',squeeze=True,index_col=0)

In [47]:
def gbif_basis_of_record_counts(gbif):
    foo = gbif.basisOfRecord
    return [foo[foo==value].count() for value in gbif_basisOfRecord_values.values]

# Création des features

In [48]:
aggregation_functions = [gbif_n_institutionCode,gbif_n_recordedBy,gbif_n_obs,gbif_n_country,gbif_max_obs_date,gbif_min_obs_date,gbif_delta_obs_date,gbif_age_last_obs,gbif_age_first_obs,gbif_n_obs_last_5y,gbif_n_obs_last_10y,gbif_n_obs_last_20y,gbif_n_obs_last_50y]
aggregation_functions_coord = [gbif_min_latitude,gbif_max_latitude,gbif_mean_latitude,gbif_mean_pos_latitude,gbif_mean_neg_latitude,gbif_median_latitude,gbif_n_tropical_latitudes,gbif_eoo,gbif_aoo]

In [49]:
column_names = ['gbif_n_institutionCode','gbif_n_recordedBy','gbif_n_obs','gbif_n_country','gbif_max_obs_date','gbif_min_obs_date','gbif_delta_obs_date','gbif_age_last_obs','gbif_age_first_obs','gbif_n_obs_last_5y','gbif_n_obs_last_10y','gbif_n_obs_last_20y','gbif_n_obs_last_50y','gbif_min_latitude','gbif_max_latitude','gbif_mean_latitude','gbif_mean_pos_latitude','gbif_mean_neg_latitude','gbif_median_latitude','gbif_n_tropical_latitudes','gbif_eoo','gbif_aoo']+['gbif_n_'+str(year)+'_'+str(year+20) for year in list(date_hist_bins)[:-1]]+['gbif_geoclim_'+str(k) for k in range(1,20)]+['gbif_pop_density_1975','gbif_pop_density_2015','gbif_pop_density_delta']+['gbif_basisOfRecord_n_'+foo for foo in gbif_basisOfRecord_values]

Par checklist accapted_plant_name_id

In [50]:
def read_set_in_csv(x):
    if x=='set()':
        return set()
    else:
        return set(x.replace("'",'').strip('{}').split(', '))

In [51]:
checklist_accepted_gbif_taxonkey_matching = pd.read_csv('/home/joon/data/checklist-accepted-gbif-taxonkey-matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv})

In [52]:
import json
import geojson
from shapely.geometry import shape, Polygon

In [53]:
checklistdist = pd.read_csv('/home/joon/data/checklist_distribution_preprocessed.csv',index_col='plant_name_id')
checklistdist = checklistdist.area_code_l3
checklistdist.dropna(inplace=True)

In [54]:
with open("/home/joon/data/level3.geojson") as f:
    data = geojson.load(f)

In [55]:
n_regions = len(data['features'])
regions = pd.Series([])

for i in range(n_regions):
    region = data['features'][i]
    level3_cod = region['properties']['LEVEL3_COD']
    polygons = []
    if len(region['geometry']['coordinates'][0][0])==2:
        polygon = Polygon(region['geometry']['coordinates'][0])
        polygons.append(polygon)
    else:
        for cc in region['geometry']['coordinates']:
            polygon = Polygon(cc[0])
            polygons.append(polygon)
    regions[level3_cod] = polygons

In [56]:
def filter_native_points(df,polygons):
    point = Point(df.decimalLatitude,df.decimalLongitude)
    bools = np.array([polygon.contains(point) for polygon in polygons])
    return bool(np.sum(bools))

In [57]:
def gbif_based_features(accepted_plant_name_ids,taxonKeys):
    plant_name_ids = checklist.plant_name_id.loc[accepted_plant_name_ids]
    areas = checklistdist.loc[checklistdist.index.intersection(plant_name_ids)].unique()
    areas = [area.upper() for area in areas]
    polygons = [l for l in regions.loc[regions.index.intersection(areas)] if str(l) != 'nan']
    polygons = [polygon for l in polygons for polygon in l]

    selected_distribution = pd.DataFrame(columns=['countryCode','decimalLatitude','decimalLongitude','year','basisOfRecord','institutionCode','recordedBy'])
    selected_distribution_native = pd.DataFrame(columns=['countryCode','decimalLatitude','decimalLongitude','year','basisOfRecord','institutionCode','recordedBy'])

    for taxonKey in taxonKeys:
        filename = '/home/joon/data/gbif-countryCode-decimalLatitude-decimalLongitude-year-taxonKey-basisOfRecord-institutionCode-recordedBy-by-taxonkey-fusionned/'+str(taxonKey)+'.csv'
        if os.path.isfile(filename):
            foo = pd.read_csv(filename,header=None,names=['countryCode','decimalLatitude','decimalLongitude','year','basisOfRecord','institutionCode','recordedBy'])
            selected_distribution = selected_distribution.append(foo,ignore_index=False)

    selected_distribution_w_coord = selected_distribution.dropna(subset=['decimalLatitude','decimalLongitude']).query('decimalLongitude != 0.0 | decimalLatitude != 0.0')

    if selected_distribution_w_coord.shape[0] > 100000:
        selected_distribution_w_coord = selected_distribution_w_coord.sample(n=50000)

    if not selected_distribution_w_coord.empty:
        selected_distribution_native = selected_distribution_w_coord.loc[selected_distribution_w_coord.apply(lambda df: filter_native_points(df,polygons),axis=1)]

    features_row = [f(selected_distribution) for f in aggregation_functions]+[f(selected_distribution_w_coord) for f in aggregation_functions_coord]+list(gbif_date_hist(selected_distribution))+gbif_obs_bioclim(selected_distribution_w_coord)+gbif_pop_density(selected_distribution_w_coord)+gbif_basis_of_record_counts(selected_distribution_w_coord)

    dist_hist_row, dist_hist_native_row = [list(gbif_obs_geographic_distr_hist(foo)) if not foo.empty else False for foo in [selected_distribution_w_coord,selected_distribution_native]]

    return features_row, dist_hist_row, dist_hist_native_row

In [104]:
def gbif_based_features_process_genus(genus):
    features = pd.DataFrame([])
    dist_hist = pd.DataFrame([]) 
    dist_hist_native = pd.DataFrame([]) 

    checklist_from_genus = checklist[checklist.genus==genus]
    checklist_from_genus = checklist_from_genus[checklist_from_genus.index == checklist_from_genus.plant_name_id]
    accepted_plant_name_ids_from_genus = checklist_from_genus.index.unique()

    for accepted_plant_name_id in accepted_plant_name_ids_from_genus:
        taxonKeys = checklist_accepted_gbif_taxonkey_matching.loc[accepted_plant_name_id]
        features_row, dist_hist_row, dist_hist_native_row = gbif_based_features([accepted_plant_name_id],taxonKeys)

        features = features.append(pd.Series([accepted_plant_name_id]+features_row),ignore_index=True)

        if not dist_hist_row == False:
            dist_hist = dist_hist.append(pd.Series([accepted_plant_name_id]+dist_hist_row),ignore_index=True)

        if not dist_hist_native_row == False:
            dist_hist_native = dist_hist_native.append(pd.Series([accepted_plant_name_id]+dist_hist_native_row),ignore_index=True)

    if not features.empty:
        features.columns = ['accepted_plant_name_id']+column_names
        features = features.astype({**{'gbif_n_'+foo:'Int64'  for foo in ['institutionCode','recordedBy','obs','country','obs_last_5y','obs_last_10y','obs_last_20y','obs_last_50y','tropical_latitudes',]},
                        **{foo:'Int64' for foo in ['gbif_max_obs_date','gbif_min_obs_date','gbif_delta_obs_date','gbif_age_last_obs','gbif_age_first_obs']+['gbif_n_'+str(year)+'_'+str(year+20) for year in list(date_hist_bins)[:-1]]}})
        features.to_csv('/home/joon/data/gbif-based-features-'+genus+'.csv',header=False,index=False)
    if not dist_hist.empty:
        dist_hist.set_index(0,inplace=True)
        dist_hist = dist_hist.astype('int')
        dist_hist.to_csv('/home/joon/data/gbif-based-dist_hist-'+genus+'.csv',header=False,index=True)

    if not dist_hist_native.empty:
        dist_hist_native.set_index(0,inplace=True)
        dist_hist_native = dist_hist_native.astype('int')
        dist_hist_native.to_csv('/home/joon/data/gbif-based-dist_hist_native-'+genus+'.csv',header=False,index=True)

In [105]:
os.system('find /home/joon/data -type f -name \'gbif-based-features*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist_native*.csv\' | xargs rm')

from multiprocessing import Pool
with Pool(processes=11) as pool:
    pool.map(gbif_based_features_process_genus, checklist_genus_values)

pd.DataFrame([['accepted_plant_name_id']+column_names]).to_csv('/home/joon/data/gbif-based-features_column-names.csv',header=None,index=None)

#fusionner les résultats et supprimer les fichiers temporaires!
os.system('find /home/joon/data -type f -name \'gbif-based-features-*.csv\' | xargs cat /home/joon/data/gbif-based-features_column-names.csv > /home/joon/data/gbif-based-features.csv')
os.system('find /home/joon/data -type f -name \'gbif-based-features-*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist-*.csv\' | xargs cat /home/joon/data/gbif-based-dist_hist_column-names.csv > /home/joon/data/gbif-based-dist_hist.csv')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist-*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist_native-*.csv\' | xargs cat /home/joon/data/gbif-based-dist_hist_native_column-names.csv > /home/joon/data/gbif-based-dist_hist_native.csv')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist_native-*.csv\' | xargs rm')

/usr/lib/python3.8/multiprocessing/pool.py:125: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))


0

Par iucn_taxon_id

In [58]:
iucn_gbif_taxonkey_matching = pd.read_csv('/home/joon/data/iucn_gbif_taxonkey_matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv},dtype={0:'int'})

In [59]:
iucn_checklist_matching = pd.read_csv('/home/joon/data/iucn-checklist-matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv},dtype={0:'int'})

In [60]:
iucn_taxon_ids = iucn_gbif_taxonkey_matching.index.unique()

In [63]:
def gbif_based_features_for_iucn_process_split(split_nb):
    features = pd.DataFrame([])
    dist_hist = pd.DataFrame([]) 
    dist_hist_native = pd.DataFrame([]) 

    global splits

    split = splits[split_nb]

    for iucn_taxon_id in split:
        taxonKeys = iucn_gbif_taxonkey_matching.loc[iucn_taxon_id]
        accepted_plant_name_ids = iucn_checklist_matching.loc[iucn_taxon_id]
        features_row, dist_hist_row, dist_hist_native_row = gbif_based_features(accepted_plant_name_ids,taxonKeys)

        features = features.append(pd.Series([iucn_taxon_id]+features_row),ignore_index=True)

        if dist_hist_row != False:
            dist_hist = dist_hist.append(pd.Series([iucn_taxon_id]+dist_hist_row),ignore_index=True)

        if dist_hist_native_row != False:
            dist_hist_native = dist_hist_native.append(pd.Series([iucn_taxon_id]+dist_hist_native_row),ignore_index=True)

    features.columns = ['iucn_taxon_id']+column_names
    features = features.astype({'iucn_taxon_id':int,**{'gbif_n_'+foo:'Int64'  for foo in ['institutionCode','recordedBy','obs','country','obs_last_5y','obs_last_10y','obs_last_20y','obs_last_50y','tropical_latitudes']},
                            **{foo:'Int64' for foo in ['gbif_max_obs_date','gbif_min_obs_date','gbif_delta_obs_date','gbif_age_last_obs','gbif_age_first_obs']+['gbif_n_'+str(year)+'_'+str(year+20) for year in list(date_hist_bins)[:-1]]}})
    features.to_csv('/home/joon/data/gbif-based-features-for_iucn-split_'+str(split_nb)+'.csv',header=False,index=False)

    dist_hist = dist_hist.astype('int')
    if dist_hist.shape[0] > 0:
        dist_hist.set_index(0,inplace=True)
    dist_hist.to_csv('/home/joon/data/gbif-based-dist_hist-for_iucn-split_'+str(split_nb)+'.csv',header=False,index=True)

    dist_hist_native = dist_hist_native.astype('int')
    if dist_hist_native.shape[0] > 0:
        dist_hist_native.set_index(0,inplace=True)
    dist_hist_native.to_csv('/home/joon/data/gbif-based-dist_hist_native-for_iucn-split_'+str(split_nb)+'.csv',header=False,index=True)

In [64]:
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist-for_iucn*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist_native-for_iucn*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-features-for_iucn*.csv\' | xargs rm')

nb_splits = 1000
splits = np.array_split(iucn_taxon_ids,nb_splits)

from multiprocessing import Pool
with Pool(processes=11) as pool:
    pool.map(gbif_based_features_for_iucn_process_split,range(len(splits)))

pd.DataFrame([['iucn_taxon_id']+column_names]).to_csv('/home/joon/data/gbif-based-features-for_iucn-column-names.csv',header=None,index=None)

os.system('find /home/joon/data -type f -name \'gbif-based-features-for_iucn-split*.csv\' | xargs cat /home/joon/data/gbif-based-features-for_iucn-column-names.csv > /home/joon/data/gbif-based-features-for_iucn.csv')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist-for_iucn-split*.csv\' | xargs cat > /home/joon/data/gbif-based-dist_hist-for_iucn.csv')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist-for_iucn-*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist_native-for_iucn-split*.csv\' | xargs cat > /home/joon/data/gbif-based-dist_hist_native-for_iucn.csv')
os.system('find /home/joon/data -type f -name \'gbif-based-dist_hist_native-for_iucn-*.csv\' | xargs rm')
os.system('find /home/joon/data -type f -name \'gbif-based-features-for_iucn-*.csv\' | xargs rm')

/usr/lib/python3.8/multiprocessing/pool.py:125: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))


0